In [1]:
import numpy as np

import os
import torch
from torch.utils.data import TensorDataset, DataLoader
import warnings
import pickle
warnings.filterwarnings("ignore")

from src.data_assemble.assemble_ml import *
from src.data_assemble.assemble_conv import *
from src.models.utils import *

/home/lukashevich/miniconda3/envs/pygdal/lib/python3.5/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
feature_names = [['tasmax', 'tasmin', 'pr'], ['elevation']]
path_to_tifs = ['data/history/*.tif', 'data/elev/*.tif']
loaded_arr = np.loadtxt("wind_in_box.txt")  
# This loadedArr is a 2D array, therefore, we need to convert it to the original array shape.reshaping to get original matrice with original shape.
cmip = loaded_arr.reshape(3652, 15, 22)
blocks = make_blocks(feature_names, path_to_tifs, cmip=cmip, half_side_size = 3, dset_num = 0)

Reading from .tifs
.tifs has been read


100%|██████████| 4/4 [00:00<00:00, 7253.44it/s]


In [3]:
start = '2006-01-01'
end = '2016-01-01'
df = pd.read_csv('data_meteo_kk.csv')
station_list = pd.read_csv('weatherstation_list.csv')

In [4]:
station_names = ['Анапа', 'Армавир', 'Краснодар, Круглик', 'Сочи', 'Туапсе', 'Приморско-Ахтарск', 'Красная Поляна']
stations_pixs = get_pixel_stations(path_to_tifs[0], feature_names[0], station_names, station_list)

In [5]:
target = get_y(df, start, end, speed_th=20)

In [7]:
X, y = assemble_numpy_ds(blocks, target, stations_pixs)

100%|██████████| 7/7 [00:00<00:00, 378.16it/s]


In [14]:
path_to_dump = os.path.join('data','nn_data')
trg_path = os.path.join(path_to_dump, 'target')
obj_path   = os.path.join(path_to_dump, 'objects')
if not os.path.isdir(trg_path):
    os.makedirs(trg_path)
if not os.path.isdir(obj_path):
    os.makedirs(obj_path)
with open(os.path.join(obj_path, 'objects.pkl'),'wb') as f:
    pickle.dump(X, f)
with open(os.path.join(trg_path, 'target.pkl'),'wb') as f:
    pickle.dump(y, f)

In [9]:
X_tensor = torch.tensor(X).cuda()
y_tensor = torch.tensor(y).cuda()
my_dataset = TensorDataset(X_tensor,y_tensor) # create your datset
my_dataloader = DataLoader(my_dataset) # create your dataloader

AssertionError: Torch not compiled with CUDA enabled